In [1]:
import spacy
from spacy import displacy

In [22]:
nlp = spacy.load('en_core_web_sm')

In [32]:
doc = nlp(u'the procedure for issuing visas to Ukraine')
displacy.render(doc, style='dep', jupyter=True)

In [5]:
x=doc[3]
x.pos_

'ADP'